In [466]:
import schedule
import time
import requests
import pandas as pd
from datetime import datetime, timedelta, date
import pyodbc
from sqlalchemy import create_engine
import mysql.connector
import csv

In [467]:
db_config = {
    'host': 'Localhost',
    'user': 'root',
    'password': '8november2002',
    'database': 'bhavDB',
}
connection = mysql.connector.connect(**db_config)
cursor = connection.cursor()
# engine = create_engine(f"mysql+mysqlconnector://{db_config['user']}:{db_config['password']}@{db_config['host']}/{db_config['database']}")

In [468]:
data=[]
date="07022024"
current_date=datetime.strptime(date,"%d%m%Y") #25-01-2024
counter=0
while(True):
    #Check response and append data
    query=f"SELECT * FROM StockData WHERE DATE1 = '{current_date}'"
    cursor.execute(query)
    result= cursor.fetchall()
    if(len(result)!=0):
        data.append(result)
        counter+=1
        print(f"Open {current_date}")
    else:
        print(f"Close {current_date}")
    current_date-=timedelta(days=1)
    #Break Loop
    if(counter==3):
        break
print(data)

Open 2024-02-07 00:00:00
Open 2024-02-06 00:00:00
Open 2024-02-05 00:00:00
[[('20MICRONS', 'EQ', datetime.date(2024, 2, 7), 155.25, 155.25, 165.0, 155.25, 160.6, 160.9, 161.71, 149969, 242.52, 8393, '72449', '48.31'), ('21STCENMGM', 'BE', datetime.date(2024, 2, 7), 41.25, 42.05, 42.05, 42.05, 42.05, 42.05, 42.05, 24963, 10.5, 60, '-', '-'), ('360ONE', 'EQ', datetime.date(2024, 2, 7), 628.5, 639.9, 652.7, 632.05, 639.0, 643.1, 640.56, 321124, 2056.99, 24642, '212570', '66.20'), ('3IINFOLTD', 'EQ', datetime.date(2024, 2, 7), 58.4, 58.8, 61.2, 57.8, 59.5, 59.7, 59.55, 4167456, 2481.56, 14946, '1292411', '31.01'), ('3MINDIA', 'EQ', datetime.date(2024, 2, 7), 33123.2, 33500.0, 34000.0, 33320.3, 33985.0, 33972.6, 33910.8, 7492, 2540.59, 2483, '5136', '68.55'), ('3PLAND', 'BE', datetime.date(2024, 2, 7), 38.35, 39.0, 39.4, 37.0, 38.0, 38.15, 38.27, 31031, 11.88, 137, '-', '-'), ('5PAISA', 'EQ', datetime.date(2024, 2, 7), 598.05, 616.95, 616.95, 598.0, 602.9, 599.65, 602.78, 52657, 317.4, 4465

In [469]:
column_names = [desc[0] for desc in cursor.description]
print(column_names)

['SYMBOL', 'SERIES', 'DATE1', 'PREV_CLOSE', 'OPEN_PRICE', 'HIGH_PRICE', 'LOW_PRICE', 'LAST_PRICE', 'CLOSE_PRICE', 'AVG_PRICE', 'TTL_TRD_QNTY', 'TURNOVER_LACS', 'NO_OF_TRADES', 'DELIV_QTY', 'DELIV_PER']


In [470]:
connection.commit()
cursor.close()
connection.close()

In [471]:
df_today = pd.DataFrame(data[0], columns=column_names)
df_prev = pd.DataFrame(data[1], columns=column_names)
df_prev2 = pd.DataFrame(data[2], columns=column_names)

In [472]:
import pandas as pd
def greenSPY(df_today, df_prev, df_prev2):
    symbols = df_today['SYMBOL'].unique().tolist()

    for symbol in symbols:
        today = df_today[df_today['SYMBOL'] == symbol]
        #print(today)
        prev_day = df_prev[df_prev['SYMBOL'] == symbol]
        #print(prev_day)
        prev_day2 = df_prev2[df_prev2['SYMBOL'] == symbol]
        #print(prev_day2)

        # Check if any of the filtered DataFrames is empty
        if today.empty or prev_day.empty or prev_day2.empty:
            continue

        today = today.iloc[0]
        second_day = prev_day.iloc[0]
        first_day = prev_day2.iloc[0]

        # Apply condition to find colors of all three days candles
        second_day_color = 'red' if second_day['OPEN_PRICE'] > second_day['CLOSE_PRICE'] else 'green'
        first_day_color = 'green' if first_day['OPEN_PRICE'] < first_day['CLOSE_PRICE'] else 'red'
        today_color = 'green' if today['OPEN_PRICE'] < today['CLOSE_PRICE'] else 'red'

        # Condition check
        condition_met = (
            first_day_color == 'red' and
            second_day_color == 'green' and
            today_color == 'green' and
            second_day['HIGH_PRICE'] < first_day['OPEN_PRICE'] and
            second_day['LOW_PRICE'] > first_day['CLOSE_PRICE'] and
            today['CLOSE_PRICE'] > first_day['HIGH_PRICE']
        )
        # print(f"For Symbol: {symbol, condition_met}")
        # Check if the condition is met and proceed
        if condition_met:
            # entry_price = today['OPEN_PRICE']
            stop_loss = first_day['LOW_PRICE']
            # target_price = entry_price * 1.1

            print(f"For Symbol: {symbol}")
            # print(f"Buy Signal - Entry Price:  Tomorrow Open Price")
            print(f"Stop Loss: {stop_loss}")
            # print(f"Target Price: entry_price * 1.1")
            print()
            filename = 'greenSPY.csv'
            with open(filename, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                date_object = datetime.strptime(date, '%d%m%Y')
                csvwriter.writerows([[symbol,date_object.strftime('%d-%b-%Y')]])

In [473]:
greenSPY(df_today, df_prev, df_prev2)

For Symbol: CANFINHOME
Stop Loss: 797.0

For Symbol: ESABINDIA
Stop Loss: 5349.0

For Symbol: LAOPALA
Stop Loss: 351.6

For Symbol: MARSHALL
Stop Loss: 38.55

For Symbol: PGEL
Stop Loss: 1924.5

For Symbol: VIJAYA
Stop Loss: 608.55



In [474]:
import pandas as pd

def redSPY(df_today, df_prev, df_prev2):

    symbols = df_today['SYMBOL'].unique().tolist()

    for symbol in symbols:
        today = df_today[df_today['SYMBOL'] == symbol]
        #print(today)
        prev_day = df_prev[df_prev['SYMBOL'] == symbol]
        #print(prev_day)
        prev_day2 = df_prev2[df_prev2['SYMBOL'] == symbol]
        #print(prev_day2)

        # Check if any of the filtered DataFrames is empty
        if today.empty or prev_day.empty or prev_day2.empty:
            continue

        today = today.iloc[0]
        second_day = prev_day.iloc[0]
        first_day = prev_day2.iloc[0]

        # Apply condition to find colors of all three days candles
        second_day_color = 'red' if second_day['OPEN_PRICE'] > second_day['CLOSE_PRICE'] else 'green'
        first_day_color = 'green' if first_day['OPEN_PRICE'] < first_day['CLOSE_PRICE'] else 'red'
        today_color = 'green' if today['OPEN_PRICE'] < today['CLOSE_PRICE'] else 'red'

        # Condition check
        condition_met = (
            first_day_color == 'green' and
            second_day_color == 'red' and
            today_color == 'red' and
            second_day['LOW_PRICE'] > first_day['OPEN_PRICE'] and
            second_day['HIGH_PRICE'] < first_day['CLOSE_PRICE'] and
            today['CLOSE_PRICE'] < first_day['LOW_PRICE']
        )
        # print(f"For Symbol: {symbol, condition_met}")
        # Check if the condition is met and proceed
        if condition_met:
            # entry_price = today['OPEN_PRICE']
            stop_loss = first_day['HIGH_PRICE']
            # target_price = entry_price * 1.1

            print(f"For Symbol: {symbol}")
            # print(f"Sell Signal - Entry Price: Tomorrow Open Price")
            print(f"Stop Loss: {stop_loss}")
            # print(f"Target Price: entry_price * 1.1")
            print()
            filename = 'redSPY.csv'
            with open(filename, 'a', newline='') as csvfile:
                csvwriter = csv.writer(csvfile)
                date_object = datetime.strptime(date, '%d%m%Y')
                csvwriter.writerows([[symbol,date_object.strftime('%d-%b-%Y')]])

In [475]:
redSPY(df_today, df_prev, df_prev2)